# Finding the hip joint centre using the Functional method

> Alex Woodall, Thor Besier <br>
> Auckland Bioengineering Institute <br>
> University of Auckland, New Zealand

There are three categories of methods which are used to describe joint kinematics: <br>

* __Antomical__ - describe the joint using anatomical landmarks
* __Functional__ - describe the joint using a mathematical model 
* __Optimisation__ - adjust joint parameters or model degrees of freedom to match experimental data

The hip centre is used to describe the femur coordinate system. Gamage and Lasenby (2002) found that optimisation methods using a functional method __improves prediction error__.

There are, however, a few caveats with the functional hip method.
* It works well in subjects who can generate a large range of motion at the hip (>60° in flexion/extension and >30° abduction)
* Does not remove errors from soft tissue artefact
* Adds time to your motion capture protocol
* May not work so well in patients with severe pathology or bone deformity (e.g., children with Cerebral Palsy)

We will be using the functional hip method to find the hip joint centre.

Start by importing the libraries needed to run the task.

In [1]:
# Import the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib notebook

from read_trc import read_trc
from segment_orientation_V1V3 import segment_orientation_V1V3
from calc_HJC import calc_HJC

## Reading in a TRC file

We can read in the desired trc file, containing the coordinate data for each of the markers during a motion capture recording session.

In [25]:
mkr_data, _ = read_trc()

In [26]:
for key,value in mkr_data["Data"]["Markers"].items():
    print(key)

RAsis
LAsis
RPsis
LPsis
LThighSuperior
LThighInferior
LThighLateral


## Choosing the markers

To calculate the right hip joint centre, we need the markers on the hip, as well as the technical markers on the right thigh. This allows us to measure the centre of rotation.

The hip markers also allow us to define a marker for the sacrum (half way between the LPsis and RPsis) and the pelvis origin (half way between the LAsis and RAsis). If there is a sacrum marker in the marker set, feel free to use that instead of calculating it.

In [23]:
LASI = mkr_data["Data"]["Markers"]["LAsis"]["All"]
RASI = mkr_data["Data"]["Markers"]["RAsis"]["All"]
LPSI = mkr_data["Data"]["Markers"]["LPsis"]["All"]
RPSI = mkr_data["Data"]["Markers"]["RPsis"]["All"]

TH1 = mkr_data["Data"]["Markers"]["RThighSuperior"]["All"]
TH2 = mkr_data["Data"]["Markers"]["RThighInferior"]["All"]
TH3 = mkr_data["Data"]["Markers"]["RThighLateral"]["All"]

KeyError: 'RThighSuperior'

In [14]:
# Define sacrum marker
SACR = (LPSI + RPSI)/2

# Define the pelvis origin
origin_pelvis = (LASI + RASI)/2

## Calculating unit vectors

The functional method requires us to calculate the unit vectors of the pelvis segment relative to the global coordinate system. For more information on this, see the _segment_orientation_V1V3()_ function.

In [15]:
# Calculate unit vectors of pelvis segment relative to global
e1_pelvis, e2_pelvis, e3_pelvis = segment_orientation_V1V3(origin_pelvis - SACR, RASI - LASI)

TH1_pelvis = []
TH2_pelvis = []
TH3_pelvis = []

## Pelvis coordinate system

To calculate the hip joint centre, we need to transform the marker points into the pelvis coordinate system.

In [16]:
# Transform the marker points into the pelvis coordinate system
for i in range(len(e1_pelvis)):
    rotation_matrix = np.array([e1_pelvis[i,:], e2_pelvis[i,:], e3_pelvis[i,:]])
    origin_pelvis_vector = np.transpose(origin_pelvis[i,:])

    TH1_pelvis.append(np.matmul(rotation_matrix,TH1[i,:]) - np.matmul(rotation_matrix,origin_pelvis_vector))
    TH2_pelvis.append(np.matmul(rotation_matrix,TH2[i,:]) - np.matmul(rotation_matrix,origin_pelvis_vector))
    TH3_pelvis.append(np.matmul(rotation_matrix,TH3[i,:]) - np.matmul(rotation_matrix,origin_pelvis_vector))

TH1_pelvis = np.array(TH1_pelvis)
TH2_pelvis = np.array(TH2_pelvis)
TH3_pelvis = np.array(TH3_pelvis)

thigh_markers_in_pelvis_cs = np.concatenate((TH1_pelvis, TH2_pelvis, TH3_pelvis), axis=1)

## Calculating the hip joint centre

Once that's all done, we can now calculate the hip joint centre using the functional method described by Gamage and Lasenby (2002).

In [17]:
HJC_location = calc_HJC(thigh_markers_in_pelvis_cs)

print(HJC_location)

[[-91.81137159]
 [-78.21037708]
 [-86.72669738]]


## Visualisation

Now you can plot the results in the coordinate frame of the pelvis...